<a href="https://colab.research.google.com/github/v-chabaux/computer-vision/blob/main/ram_grounded_sam/ram_grounded_sam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#SRC : https://github.com/IDEA-Research/Grounded-Segment-Anything

import os
import sys

!pip install supervision
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd GroundingDINO
!pip install -e .
!mkdir weights
%cd weights
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
%cd ..
%cd ..
sys.path.append(os.path.join(os.getcwd(), "GroundingDINO"))

!pip install git+https://github.com/facebookresearch/segment-anything.git
!mkdir sam_checkpoint
%cd sam_checkpoint
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
%cd ..

!git clone https://github.com/xinyu1205/recognize-anything.git
%cd recognize-anything
!pip install -r requirements.txt
%cd ..
os.rename("recognize-anything","recognizeAnything")
sys.path.append(os.path.join(os.getcwd(), "recognizeAnything"))

!mkdir ram_checkpoint
%cd ram_checkpoint
!wget https://huggingface.co/spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth
!wget https://huggingface.co/spaces/xinyu1205/recognize-anything/resolve/main/tag2text_swin_14m.pth
!wget https://huggingface.co/xinyu1205/recognize-anything-plus-model/resolve/main/ram_plus_swin_large_14m.pth

%cd ..

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 2.6 MB/s eta 0:00:00
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 401 (delta 128), reused 120 (delta 120), pack-reused 237
Receiving objects: 100% (401/401), 12.84 MiB | 16.19 MiB/s, done.
Resolving deltas: 100% (205/205), done.
/content/GroundingDINO
Obtaining file:///content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 101.1 MB/s

In [2]:
import numpy as np
import json
import torch
import torchvision
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as TS
import supervision as sv

# Grounding DINO
from groundingdino.util.inference import Model

# segment anything
from segment_anything import build_sam, SamPredictor

# Recognize Anything Model & Tag2Text
from recognizeAnything.ram.models import ram, ram_plus, tag2text
from recognizeAnything.ram import inference_ram, inference_ram, inference_tag2text, inference_ram_openset, get_transform
from recognizeAnything.ram.utils import build_openset_label_embedding

/content/GroundingDINO/groundingdino/models/GroundingDINO/ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available() :
  device_i = "cuda"
  print("Used GPU : {}\n".format(torch.cuda.get_device_name(0)))
else:
  device_i = "cpu"
  print("No GPU available. CPU is used\n")

ram_checkpoint = "ram_checkpoint/ram_swin_large_14m.pth"
ram_model = ram(pretrained=ram_checkpoint, image_size=384, vit='swin_l')
#######set openset interference
physical_cover_elements_detective_novel = ["car", "cake", "gun","woman", "plane", "fire", "bank bill" "dollar", "money","cat","dog","house", "chair"]
openset_label_embedding, openset_categories = build_openset_label_embedding(physical_cover_elements_detective_novel)
ram_model.tag_list = np.array(openset_categories)
ram_model.label_embed = torch.nn.Parameter(openset_label_embedding.float())
ram_model.num_class = len(openset_categories)
ram_model.class_threshold = torch.ones(ram_model.num_class) * 0.65
#######
ram_model.eval()
ram_model = ram_model.to(device)

# GroundingDINO
config_file = "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
grounded_checkpoint = "GroundingDINO/weights/groundingdino_swint_ogc.pth"
grounding_dino_model = Model(model_config_path=config_file, model_checkpoint_path=grounded_checkpoint, device = device_i)

#Segment Anything
sam_checkpoint = "sam_checkpoint/sam_vit_h_4b8939.pth"
predictor = SamPredictor(build_sam(checkpoint=sam_checkpoint).to(device))

"""
# Recognize Anything / tag2text
tag2text_checkpoint = "ram_checkpoint/tag2text_swin_14m.pth"
tag2text_model = tag2text(pretrained=tag2text_checkpoint, image_size=384, vit='swin_b')
tag2text_model.eval()
tag2text_model = tag2text_model.to(device)

ram_plus_checkpoint = "ram_checkpoint/ram_plus_swin_large_14m.pth"
ram_plus_model = ram_plus(pretrained=ram_plus_checkpoint, image_size=384, vit='swin_l')
ram_plus_model.eval()
ram_plus_model = ram_plus_model.to(device)
"""

No GPU available. CPU is used



/encoder/layer/0/crossattention/self/query is tied
/encoder/layer/0/crossattention/self/key is tied
/encoder/layer/0/crossattention/self/value is tied
/encoder/layer/0/crossattention/output/dense is tied
/encoder/layer/0/crossattention/output/LayerNorm is tied
/encoder/layer/0/intermediate/dense is tied
/encoder/layer/0/output/dense is tied
/encoder/layer/0/output/LayerNorm is tied
/encoder/layer/1/crossattention/self/query is tied
/encoder/layer/1/crossattention/self/key is tied
/encoder/layer/1/crossattention/self/value is tied
/encoder/layer/1/crossattention/output/dense is tied
/encoder/layer/1/crossattention/output/LayerNorm is tied
/encoder/layer/1/intermediate/dense is tied
/encoder/layer/1/output/dense is tied
/encoder/layer/1/output/LayerNorm is tied
--------------
ram_checkpoint/ram_swin_large_14m.pth
--------------
load checkpoint from ram_checkpoint/ram_swin_large_14m.pth
vit: swin_l
Creating pretrained CLIP model


100%|███████████████████████████████████████| 335M/335M [00:09<00:00, 38.4MiB/s]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


'\n# Recognize Anything / tag2text\ntag2text_checkpoint = "ram_checkpoint/tag2text_swin_14m.pth"\ntag2text_model = tag2text(pretrained=tag2text_checkpoint, image_size=384, vit=\'swin_b\')\ntag2text_model.eval()\ntag2text_model = tag2text_model.to(device)\n\nram_plus_checkpoint = "ram_checkpoint/ram_plus_swin_large_14m.pth"\nram_plus_model = ram_plus(pretrained=ram_plus_checkpoint, image_size=384, vit=\'swin_l\')\nram_plus_model.eval()\nram_plus_model = ram_plus_model.to(device)\n'

In [4]:
image_local_path = "image_667.jpg"
transform = get_transform(image_size=384)
image = transform(Image.open(image_local_path)).unsqueeze(0).to(device)

In [5]:
#RAM openset inference
res_ram = inference_ram_openset(image , ram_model)
print("RAM Tags: ", res_ram)

"""
#RAM++ inference
res_ram_plus = inference_ram(image , ram_plus_model)
print("RAM++ Tags: ", res_ram_plus[0])

#RAM inference
res_ram = inference_ram(image , ram_model)
print("RAM Tags: ", res_ram[0])

#Tag2text inference
res_tag2text = inference_tag2text(image , tag2text_model)
print("Tag2text Tags: ", res_tag2text[0])
"""

RAM Tags:  woman | plane | fire | money


'\n#RAM++ inference\nres_ram_plus = inference_ram(image , ram_plus_model)\nprint("RAM++ Tags: ", res_ram_plus[0])\n\n#RAM inference\nres_ram = inference_ram(image , ram_model)\nprint("RAM Tags: ", res_ram[0])\n\n#Tag2text inference\nres_tag2text = inference_tag2text(image , tag2text_model)\nprint("Tag2text Tags: ", res_tag2text[0])\n'

In [ ]:
# Predict classes and hyper-param for GroundingDINO

CLASSES = [i.strip() for i in res_ram.split("|")]

BOX_THRESHOLD = 0.2
TEXT_THRESHOLD = 0.2
NMS_THRESHOLD = 0.3

if not os.path.exists('output'):
   os.makedirs('output')

# load image
image_for_seg = cv2.imread(image_local_path)

# detect objects
detections = grounding_dino_model.predict_with_classes(
    image=image_for_seg,
    classes=CLASSES,
    box_threshold=BOX_THRESHOLD,
    text_threshold=BOX_THRESHOLD
)

# annotate image with detections
box_annotator = sv.BoxAnnotator()
labels = [
    f"{CLASSES[class_id]} {confidence:0.2f}"
    for _, _, confidence, class_id, _
    in detections]
annotated_frame = box_annotator.annotate(scene=image_for_seg.copy(), detections=detections, labels=labels)

# save the annotated grounding dino image
cv2.imwrite("output/groundingdino_annotated_image.jpg", annotated_frame)


# NMS post process
print(f"Before NMS: {len(detections.xyxy)} boxes")
nms_idx = torchvision.ops.nms(
    torch.from_numpy(detections.xyxy),
    torch.from_numpy(detections.confidence),
    NMS_THRESHOLD
).numpy().tolist()

detections.xyxy = detections.xyxy[nms_idx]
detections.confidence = detections.confidence[nms_idx]
detections.class_id = detections.class_id[nms_idx]

print(f"After NMS: {len(detections.xyxy)} boxes")

# Prompting SAM with detected boxes
def segment(sam_predictor: SamPredictor, image: np.ndarray, xyxy: np.ndarray) -> np.ndarray:
    sam_predictor.set_image(image_for_seg)
    result_masks = []
    for box in xyxy:
        masks, scores, logits = sam_predictor.predict(
            box=box,
            multimask_output=True
        )
        index = np.argmax(scores)
        result_masks.append(masks[index])
    return np.array(result_masks)


# convert detections to masks
detections.mask = segment(
    sam_predictor=predictor,
    image=cv2.cvtColor(image_for_seg, cv2.COLOR_BGR2RGB),
    xyxy=detections.xyxy
)

# annotate image with detections
box_annotator = sv.BoxAnnotator()
mask_annotator = sv.MaskAnnotator()
labels = [
    f"{CLASSES[class_id]} {confidence:0.2f}"
    for _, _, confidence, class_id, _
    in detections]
annotated_image = mask_annotator.annotate(scene=image_for_seg.copy(), detections=detections)
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)

# save the annotated grounded-sam image
cv2.imwrite("output/grounded_sam_annotated_image.jpg", annotated_image)

Before NMS: 4 boxes
After NMS: 4 boxes


True